In [1]:
from google import genai
from pydantic import BaseModel
import os
import json

from dotenv import load_dotenv

load_dotenv()

class City(BaseModel):
    capital: str
    area: float

llm = genai.Client(
    api_key = os.getenv("GEMINI_API_KEY"),

)

response = llm.models.generate_content(
    model="gemini-2.5-flash",
    contents="Apa ibukota saat ini di Indonesia dan luasnya?",
    config={
        "response_mime_type": "application/json",
        "response_schema": City
    }
)

print(response.text)
print("The capital of Indonesia now is " + json.loads(response.text)['capital'] + " with area " + str(json.loads(response.text)['area']))

{"capital": "Jakarta", "area": 664}
The capital of Indonesia now is Jakarta with area 664


In [6]:
from markitdown import MarkItDown
from chonkie import RecursiveChunker, Visualizer

md = MarkItDown()

source_file = "sample pdf.pdf"
result = md.convert(source_file)

markdown = result.text_content

chunker = RecursiveChunker.from_recipe("markdown", lang="en")
chunks = chunker.chunk(markdown)

viz = Visualizer()
viz.save("chonkie.html", chunks)

d:\DOWNLOAD\training_rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HTML visualization saved to: file://d:\DOWNLOAD\training_rag\chonkie.html


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
    length_function = len,
    separators = ["\n\n","\n"," ", ""]
)

load_docs = PyPDFLoader("sample pdf.pdf")
pages = load_docs.load()

chunks = text_splitter.split_documents(pages)

#print(len(pages))
#print(len(chunks))
#print(chunks[0].page_content)
#print(chunks[0].metadata)

In [3]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

try:
    client = chromadb.PersistentClient('db/workshop_1')
    client.delete_collection(name="gemini_demo")
except:
    pass

google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(
    api_key=os.getenv("GEMINI_API_KEY"),
    model_name="gemini-embedding-001"
)

client = chromadb.PersistentClient('db/workshop_1')

collection = client.create_collection(
    name = "gemini_demo",
    embedding_function = google_ef
)

for filename in os.listdir("D:/DOWNLOAD/training_rag"):
    if filename.endswith("pdf"):
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 500,
            chunk_overlap = 100,
            length_function = len,
            separators = ["\n\n","\n"," ", ""]
        )
        file = os.path.join("D:/DOWNLOAD/training_rag", filename)
        load_docs = PyPDFLoader(file)
        pages = load_docs.load()

        chunks = text_splitter.split_documents(pages)
        collection.add(
            ids = [f"chunk_{i}" for i, _ in enumerate(chunks)],
            documents = [doc.page_content for doc in chunks],
            metadatas = [doc.metadata for doc in chunks]
        )

In [21]:
collection = client.get_collection("gemini_demo", embedding_function=google_ef)

query = "Di binus ada makanan apa aja?"

results = collection.query(
    query_texts = query,
    n_results = 5
)

print(results['documents'][0][4])

 Financial Ruin: The cost of alcohol, combined with potential job loss due to 
poor performance or legal fees (DUI), can lead to bankruptcy. 
 Legal Consequences: Arrests for public intoxication, disorderly conduct, or 
driving under the inﬂuence (DUI) leave permanent criminal records. 
 
5. Summary of Statistics 
 Fatalities: Alcohol contributes to more than 3 million deaths globally each year 
(WHO). 
 Youth Risk: Alcohol is a leading factor in death for people aged 15–49.


In [22]:
context = "\n".join(results['documents'][0])

prompt = f"""Use the following context to answer the question. If you cannot answer the question based on the context, say "I cannot answer this based on the provided context."

Context:
{context}

Question: {query}"""

llm = genai.Client(
    api_key = os.getenv("GEMINI_API_KEY")
)

response = llm.models.generate_content(
    model = "gemini-2.5-flash",
    contents = prompt,
    config = {
        "system_instruction": "You are helpful assistant that answer the question based on provided context",
        "temperature":"0" #0 - 1
    }
)

print(response.text)

I cannot answer this based on the provided context.


In [1]:
!pip freeze > requirements.txt

In [17]:
import duckdb

db = duckdb.connect("db/workshop_db")
table_name = 'alcohol_price'

if not db.sql(f"SELECT * FROM information_schema.tables WHERE table_name = '{table_name}'").fetchone():
    db.sql(f"CREATE TABLE {table_name} AS SELECT * FROM read_csv_auto('sample csv.csv')")

df = db.sql(f"DESCRIBE {table_name}")

query = "Alcohol yang paling mahal dong apa ya?"

prompt = f"""
    You are a helpful assistant that can generate SQL queries that answer the question about the data.
    Please do select it from table {table_name}
    Also do notes that the column of the {table_name} were:
    {df}

    Here are the question for you to answer: {query}
"""

response = llm.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config = {
        "system_instruction": "You are a helpful assistant that can generate SQL queries. Only return the SQL and nothing more",
        "temperature": "0"
    }
)

sql_query = response.text
sql_query = sql_query.replace("```sql","").replace("```","")
results = db.sql(sql_query)

In [26]:
import redis
import json

r = redis.Redis(host='localhost', port=6379)

client = genai.Client(
    api_key = os.getenv('GEMINI_API_KEY'),
)

def get_history(user_id):
    messages = r.lrange(f"chat:{user_id}",0,-1)
    return [json.loads(m.decode()) for m in messages]

def add_history(user_id, role, messages):
    r.rpush(f"chat:{user_id}", json.dumps({"role": role, "content": messages}))

def rewrite_query(query, conversations: list[dict]):
    history_context = "\n".join([f"{conv['role']}: {conv['content']}" for conv in conversations])

    prompt = f"""
    The conversation context is
    {history_context}
    Query:
    {query}
    """

    response = client.models.generate_content(
        model = "gemini-2.5-flash",
        contents = prompt,
        config = {
            "system_instruction": "You are helpful assistant that can rewrite queries to be more accurate. Always put the name of the person or produt if the had mentioned.",
            "temperature": "0"
        }
    )

    return response.text

def get_rag_answer(results, query, model, model_name="gemini-2.5-flash", temperature=0):
    context = "\n".join(results)

    prompt = f"""
    Use the following context to answer the question.
    If you cannot answer the question based on the context, please answer "I can't answer it based on the following context".
    Context:
    {context}

    Query:
    {query}
    """

    response = client.models.generate_content(
        model = f"{model_name}",
        contents = prompt,
        config = {
            "system_instruction": "You are helpful assistant that can rewrite queries to be more accurate. Always put the name of the person or produt if the had mentioned.",
            "temperature": f"{temperature}"
        }
    )

    return response.text

def retrieve_query(query):
    results = collection.query(
        query_texts = query,
        n_results = 3
    )

    return get_rag_answer(results['documents'][0], query, client)

def text_2_db(query):
    db = duckdb.connect("db/workshop_db")
    table_name = 'alcohol_price'

    df = db.sql(f"DESCRIBE {table_name}")

    prompt = f"""
        You are a helpful assistant that can generate SQL queries that answer the question about the data.
        Please do select it from table {table_name}
        Also do notes that the column of the {table_name} were:
        {df}

        Here are the question for you to answer: {query}
    """

    response = llm.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
        config = {
            "system_instruction": "You are a helpful assistant that can generate SQL queries. Only return the SQL and nothing more",
            "temperature": "0"
        }
    )

    sql_query = response.text
    sql_query = sql_query.replace("```sql","").replace("```","")

    res = db.sql(sql_query)
    return res

def classify_intent(query):
    prompt = f"""
    Decide if the query needs a vector search or text 2 sql search.print
    If the user is asking for specific data which is usually like a numerical operation like greater than, less than, etc. then it is a text 2 sql search. Similary if they are asking for details mentioning a specific field name like find me details of employee with id 123 then it is a text 2 sql search.
    If the user query is a general, hi, hello, etc. then it is a General intent.
    For everything else, it is a vector search.

    Here are the list of tables in the database:
    {db.sql("SELECT table_name FROM information_schema.tables").fetchall()}

    here is the list of pdf documents we have ingested in vector db:
    "sample pdf.pdf" regarding the danger of alcohol
    "sample pdf 1.pdf" regarding pros and cons of alcohol
    "sample pdf 2.pdf" regarding health concious guide of alcohol

    The query is: {query}
    Only return the Intent as a string. It will be either "vector" or "text_2_db" or "general".
    """

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents = prompt,
        config = {
            "temperature": "0"
        }
    )

    return response.text

def run_agentic_rag(query, user_id):
    add_history(user_id, "user", query)
    rewrite = rewrite_query(query, get_history(user_id))
    print(rewrite)
    intent = classify_intent(rewrite)
    print(intent)
    if intent == "general":
        return "No intention, can't answer question"
    elif intent == "text_2_db":
        return text_2_db(rewrite)
    elif intent == "vector":
        return retrieve_query(rewrite)

res = run_agentic_rag("Alokohol paling mahal apa", "a")
print(res)

Alkohol paling mahal apa
text_2_db


ConnectionException: Connection Error: Connection has already been closed